In [5]:
!pip install langchain-astradb
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langch

In [34]:
!pip install langchain_groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain_groq]


In [35]:
import os
from getpass import getpass
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import load_dataset
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [9]:
os.environ["ASTRA_DB_API_ENDPOINT"] = "https://43b50bc3-a525-4afb-adfd-3581a6768632-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "AstraCS:MJUYSyYGioSHvLYSePgGJCbG:a6571d8b382bbf9b1fa9b5787658fac5a7cfc59298ed41073334e7b3d8907c35"
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")

Enter your Groq API Key:  ········


In [10]:
embeddings = HuggingFaceEmbeddings(model_name = "myrkur/sentence-transformer-parsbert-fa-2.0")
vstore = AstraDBVectorStore(
    collection_name = "test",
    embedding = embeddings,
    token = os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
)
print("Astra vector store configured")

Astra vector store configured


In [13]:
ds = load_dataset("MaralGPT/persian_quotes")["train"]
print(ds[0])

{'body': 'بشریت باید جنگ را ریشه کن کند، پیش از آنکه جنگ، بشریت را ریشه کن کند.', 'author': 'جان اف کندی'}


In [15]:
docs = []
for i in ds:
    metadata = {"author": i["author"]}
    doc = Document(page_content = i["body"], metadata = metadata)
    docs.append(doc)

In [16]:
docs[0]

Document(metadata={'author': 'جان اف کندی'}, page_content='بشریت باید جنگ را ریشه کن کند، پیش از آنکه جنگ، بشریت را ریشه کن کند.')

In [17]:
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents")


Inserted 739 documents


In [36]:
retriever = vstore.as_retriever(search_kwargs = {"k": 3})

In [40]:
prompt = ChatPromptTemplate.from_template(
    """
    فقط بر اساس متن ارائه شده، به سوال پاسخ بده. اگر پاسخ را نمی دانی، بگو نمی دانم.
    متن: {context}
    سوال: {question}
    """
)

In [38]:
model = ChatGroq(
    model = "llama-3.3-70b-versatile",
    groq_api_key = os.getenv("GROQ_API_KEY"),
    temperature = 0
)

In [41]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
print(chain.invoke("در متن ارائه شده، بیشترین دغدغه نویسندگان چیست؟"))

بیشترین دغدغه نویسندگان در متن ارائه شده، درک و فهم درست از مفاهیم و نوشته‌ها است. حکیم عمر خیام به تلاش برای قرار دادن یک سر بزرگ بر تنه انسان‌های کوچک اشاره می‌کند، مارک تواین به درک و فهم بخش‌های مختلف انجیل اشاره می‌کند و آرتور شوپنهاور به اهمیت خواندن دوبار یک کتاب برای درک بهتر و فهمیدن قسمت‌های مختلف آن تأکید می‌کند.


In [46]:
print(chain.invoke("نیچه وظیفه انسان نمو یافته را چه می داند؟"))

نیچه وظیفه انسان نمو یافته یا "ابَر انسان" را عشق به سرنوشت خود و چیره شدن بر نیاز به هدفی ماورایی می‌داند. او معتقد است که انسانی که به این مرحله برسد، به ابَر انسان بدل خواهد شد و نماینده مرحله والاتر تکامل بشری است.


In [48]:
print(chain.invoke("آزادی مستلزم چیست؟"))

آزادی مستلزم مسئولیت است.


In [49]:
print(chain.invoke("راز موفقیت در چیست؟"))

راز موفقیت در این است که آن را سخاوتمندانه با دیگران تقسیم کنید.


In [50]:
print(chain.invoke("جمله زیر را چه کسی گفته است:\nمرا، رنگ به‌چه‌کار آید، وقتی نمی‌دانم چه‌چیزی را نقاشی کنم."))

این جمله را پابلو پیکاسو گفته است.
